### Di seguito tutte le funzioni originali del modulo AlphaStructures, per poter comparare i loro tempi di esecuzione con le nuove versioni delle stesse funzioni

In [117]:
using LinearAlgebraicRepresentation
using Delaunay, Triangle
using Combinatorics, DataStructures
using Distributed, SharedArrays
Lar = LinearAlgebraicRepresentation
include("../src/AlphaStructures.jl") # nuovo modulo AlphaStructures

Main.AlphaStructures

In [118]:
function findCenter(P::Lar.Points)::Array{Float64,1}
    dim, n = size(P)
    @assert n > 0		"findCenter: at least one points is needed."
    @assert dim >= n-1	"findCenter: Too much points"

    @assert dim < 4		"findCenter: Function not yet Programmed."

    if n == 1
        center = P[:, 1]

    elseif n == 2
        #for each dimension
        center = (P[:, 1] + P[:, 2]) / 2

    elseif n == 3
        #https://www.ics.uci.edu/~eppstein/junkyard/circumcenter.html
        if dim == 2
            denom = 2 * Lar.det([ P[:, 2] - P[:, 1]  P[:, 3] - P[:, 1] ])
            deter = (P[:, 2] - P[:, 1]) * Lar.norm(P[:, 3] - P[:, 1])^2 -
                    (P[:, 3] - P[:, 1]) * Lar.norm(P[:, 2] - P[:, 1])^2
            numer = [- deter[2], deter[1]]
            center = P[:, 1] + numer / denom

        elseif dim == 3
            #circumcenter of a triangle in R^3
            numer = Lar.norm(P[:, 3] - P[:, 1])^2 * Lar.cross(
                        Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]),
                        P[:, 2] - P[:, 1]
                    ) +
                    Lar.norm(P[:, 2] - P[:, 1])^2 * Lar.cross(
                        P[:, 3] - P[:, 1],
                        Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1]
                    )
            )
            denom = 2 * Lar.norm(
                Lar.cross(P[:, 2] - P[:, 1], P[:, 3] - P[:, 1])
            )^2
            center = P[:, 1] + numer / denom
        end

    elseif n == 4 #&& dim = 3
        # https://people.sc.fsu.edu/~jburkardt/presentations
        #	/cg_lab_tetrahedrons.pdf
        # page 6 (matrix are transposed)
        α = Lar.det([P; ones(1, 4)])
        sq = sum(abs2, P, dims = 1)
        Dx = Lar.det([sq; P[2:2,:]; P[3:3,:]; ones(1, 4)])
        Dy = Lar.det([P[1:1,:]; sq; P[3:3,:]; ones(1, 4)])
        Dz = Lar.det([P[1:1,:]; P[2:2,:]; sq; ones(1, 4)])
        center = [Dx; Dy; Dz]/2α
    end

    return center
end

findCenter (generic function with 1 method)

In [119]:
function findClosestPoint(
        Psimplex::Lar.Points, P::Lar.Points;
        metric = "circumcenter"
    )::Union{Int64, Nothing}

    @assert metric ∈ ["circumcenter", "dd"] "findClosestPoint: available metrics are
        `circumcenter` and `dd`."

    simplexDim = size(Psimplex, 2)
    @assert simplexDim <= size(Psimplex, 1) "findClosestPoint: Cannot add
    another point to the simplex."

    @assert (m = size(P, 2)) != 0 "findClosestPoint: No Points in `P`."

    radlist = SharedArray{Float64}(m)
    for col = 1 : m
        r, c = findRadius([Psimplex P[:,col]], true)
        sameSign = (
            r == Inf ||
            metric != "dd" ||
            isempty(oppositeHalfSpacePoints(
                [Psimplex P[:,col]], Psimplex, c
            ))
        )
        radlist[col] = ((-1)^(1 + sameSign)) * r
    end

    radius, closestidx = findmin(radlist)

    if radius == Inf
        closestidx = nothing
    end

    return closestidx

end


findClosestPoint (generic function with 1 method)

In [120]:
function findMedian(P::Lar.Points, ax::Int64)::Float64
    xp = sort(unique(P[ax, :]))
    if length(xp) == 1
        median = xp[1]
    else
        idx = Int64(floor(length(xp)/2))
        median = (xp[idx] + xp[idx+1])/2
    end
    return median
end

findMedian (generic function with 1 method)

In [121]:
function findRadius(
        P::Lar.Points, center=false; digits=64
    )::Union{Float64, Tuple{Float64, Array{Float64,1}}}

    c = findCenter(P)
    if any(isnan, c)
        r = Inf
    else
        r = round(
            findmin([Lar.norm(c - P[:, i]) for i = 1 : size(P, 2)])[1],
            digits = digits
        )
    end
    if center
        return r, c
    end
    return r
end

findRadius (generic function with 2 methods)

In [122]:
function matrixPerturbation(
        M::Array{Float64,2};
        atol=1e-10, row = [0], col = [0]
    )::Array{Float64,2}

    if atol == 0.0
        println("Warning: no perturbation has been performed.")
        return M
    end

    if row == [0]
        row = [i for i = 1 : size(M, 1)]
    end
    if col == [0]
        col = [i for i = 1 : size(M, 2)]
    end

    N = copy(M)
    perturbation = mod.(rand(Float64, length(row), length(col)), 2*atol).-atol
    N[row, col] = M[row, col] + perturbation
    return N
end

matrixPerturbation (generic function with 1 method)

In [123]:
function oppositeHalfSpacePoints(
        P::Lar.Points,
        face::Array{Float64,2},
        point::Array{Float64,1}
    )::Array{Int64,1}

    dim, n = size(P)
    noV = size(face, 2)
    @assert dim <= 3 "oppositeHalfSpacePoints: Not yet coded."
    @assert noV == dim "oppositeHalfSpacePoints:
        Cannot determine opposite to non hyperplanes."
    if dim == 1
        threshold = face[1]
        if point[1] < threshold
            opposite = [i for i = 1 : n if P[1, i] > threshold]
        else
            opposite = [i for i = 1 : n if P[1, i] < threshold]
        end
    elseif dim == 2
        if (Δx = face[1, 1] - face[1, 2]) != 0.0
            m = (face[2, 1] - face[2, 2]) / Δx
            q = face[2, 1] - m * face[1, 1]
            # false = under the line, true = over the line
            @assert point[2] ≠ m * point[1] + q "oppositeHalfSpacePoints,
                the point belongs to the face"
            side = sign(m * point[1] + q - point[2])
            opposite =
                [i for i = 1 : n if side * (m * P[1, i] + q - P[2, i]) < 0]
        else
            q = face[1, 1]
            side = sign(point[1] - q)
            opposite = [i for i = 1 : n if side * (P[1, i] - q) < 0]
        end


    elseif dim == 3
        axis = Lar.cross(
            face[:, 2] - face[:, 1],
            face[:, 3] - face[:, 1]
        )
        off = Lar.dot(axis, face[:, 1])
        position = Lar.dot(point, axis)
        if position < off
            opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) > off]
        else
            opposite = [i for i = 1:size(P, 2) if Lar.dot(P[:,i], axis) < off]
        end
    end

    return [
        i for i in opposite
        if sum([P[:, i] == face[:, j] for j = 1 : noV]) == 0
    ]
end

oppositeHalfSpacePoints (generic function with 1 method)

In [124]:
function planarIntersection(
        P::Lar.Points,
        face::Array{Int64,1},
        axis::Int64,
        off::Float64
    )::Int64

    pos = [P[axis, i] > off for i in face]

    if sum([P[axis, i] == off for i in face]) == length(pos)
        position = 0 # face coplanar with axis
    elseif sum(pos) == 0
        position = -1
    elseif sum(pos) == length(pos)
        position = +1
    else
        position = 0
    end

    return position
end

planarIntersection (generic function with 1 method)

In [125]:
function simplexFaces(σ::Array{Int64,1})::Array{Array{Int64,1},1}
    sort!(sort!.(collect(Combinatorics.combinations(σ, length(σ)-1))))
end

simplexFaces (generic function with 1 method)

In [126]:
function vertexInCircumball(
        P::Lar.Points,
        α_char::Float64,
        point::Array{Float64,2}
    )::Bool

    center = findCenter(P)
    return Lar.norm(point - center) <= α_char
end

vertexInCircumball (generic function with 1 method)

In [127]:
function delaunayWall(
        P::Lar.Points,
        ax = 1,
        Pblack = Float64[],
        AFL = Array{Int64,1}[],
        tetraDict = DataStructures.Dict{Array{Int64,1},Array{Float64,1}}();
        DEBUG = false
    )::Lar.Cells

    if DEBUG @show "Delaunay Wall with parameters" P ax AFL tetraDict end

    # 0 - Data Reading and Container definition
    DT = Array{Int64,1}[]		# Delaunay Triangulation
    AFLα = Array{Int64,1}[]		# (d-1)faces intersecting the Wall
    AFLplus = Array{Int64,1}[]  # (d-1)faces in positive Wall half-space
    AFLminus = Array{Int64,1}[] # (d-1)faces in positive Wall half-space
    off = findMedian(P, ax)
    if !isempty(Pblack) Pext = [P Pblack] else Pext = copy(P) end

    # 1 - Determine first simplex (if necessary)
    if isempty(AFL)
        @assert isempty(Pblack) "delaunayWall: If AFL is empty => Pblack must be"
        @assert isempty(tetraDict) "delaunayWall: If AFL is empty => tetraDict must be"
        σ = sort(firstDeWallSimplex(P, ax, off, DEBUG = DEBUG))
        push!(DT, σ)
        AFL = simplexFaces(σ)
        updateTetraDict!(P, tetraDict, AFL, σ)
    else
        @assert !isempty(Pblack) "delaunayWall: Data missing - Pblack"
        @assert !isempty(AFL) "delaunayWall: Data missing - AFL"
        @assert !isempty(tetraDict) "delaunayWall: Data missing - tetraDict"
    end

    # 2 - Build `AFL*` according to the axis `ax` with constant term `off`
    updateAFL!(
        P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG = DEBUG
    )

    # 4 - Build simplex Wall
    while !isempty(AFLα)
        # if face ∈ keys(tetraDict) oppoint = tetraDict[face]
        # else Pselection = setdiff([i for i = 1 : n], face) end
        σ = findWallSimplex(
                Pext, AFLα[1], tetraDict[AFLα[1]], size(P, 2), DEBUG = DEBUG
            )
        if σ != nothing && σ ∉ DT
            push!(DT, σ)
            AFL = simplexFaces(σ)
            updateTetraDict!(P, tetraDict, AFL, σ)
            # Split σ's Faces according in semi-spaces
            updateAFL!(
                P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG=DEBUG
            )
        else
            @assert updatelist!(AFLα, AFLα[1]) == false "delaunayWall:
                Something unespected happends while removing a face."
        end
    end

    # 5 - Change the axis `ax` and repeat until there are no faces but exposed.
    #      A.K.A. Divide & Conquer phase.
    if !isempty(AFLminus)
        union!(DT, recursiveDelaunayWall(
            P, Pblack, tetraDict, AFLminus, ax, off, false; DEBUG = DEBUG
        ))
    end
    if !isempty(AFLplus)
        union!(DT, recursiveDelaunayWall(
            P, Pblack, tetraDict, AFLplus, ax, off, true; DEBUG = DEBUG
        ))
    end

    return DT
end

delaunayWall (generic function with 5 methods)

In [128]:
function findWallSimplex(
        P::Lar.Points,
        face::Array{Int64,1},
        oppoint::Array{Float64,1},
        blackidx = size(P, 2);
        DEBUG = false
    )::Union{Array{Int64,1}, Nothing}

    if DEBUG @show "find Wall Simplex of" face oppoint end
    # Find the points in the halfspace defined by `face` that do not
    #  containsother the other point of the simplex.
    Pselection =
        oppositeHalfSpacePoints(P, P[:, face], oppoint)

    if DEBUG @show Pselection end

    # If there are no such points than the face is part of the convex hull.
    if isempty(Pselection)
        return nothing
    end

    # Find the Closest Point in the other halfspace with respect to σ
    #  according to dd-distance.
    idxbase = Pselection[ findClosestPoint(
        P[:, face], P[:, Pselection], metric = "dd"
    ) ]

    # @assert !isnothing(idxbase)
    # if isnothing(idxbase)
    # 	return nothing
    # end

    # It prevent from adding the same simplex again (cause it has been
    #  determined in a previous recursive call in the stacktrace).
    if idxbase > blackidx
        if DEBUG println("Excluding $face cause simplex already inside.") end
        return nothing
    end

    σ = sort([face; idxbase])
    if DEBUG @show "Found face" σ end

    # Check the simplex correctness
    radius, center = findRadius(P[:, σ], true)
    for i = 1 : size(P, 2)
        if Lar.norm(center - P[:, i]) < radius - 1.e-14
            # @assert i ∉ Pselection "ERROR: Numerical error
            # 	evaluating minimum radius for $σ"
            if DEBUG println("$σ discarded due to a closer point.") end
            return nothing
        end
    end

    return σ
end



findWallSimplex (generic function with 2 methods)

In [129]:
function firstDeWallSimplex(
        P::Lar.Points,
        ax::Int64,
        off::Float64;
        DEBUG = false
    )::Array{Int64,1}

    dim = size(P, 1)
    n = size(P, 2)

    if DEBUG println("Determine first Simplex with ax = $ax") end
    # the first point of the simplex is the one with coordinate `ax` maximal
    #  such that it is less than `off` (closer to α from minus)

    Pselection = findall(x -> x < off, P[ax, :])

    # it gives an error if no point are less than `off`
    #  in fact it means that all the points are located on the median,
    #  with respect to `ax`.
    @assert !isempty(Pselection) "firstDeWallSimplex: not able to build the first Delaunay
        dimplex; all the points have the same `ax` coordinate."
    newidx = Pselection[findmax(P[ax, Pselection])[2]]
    # indices will store the indices of the simplex ...
    indices = [newidx]                      #Array{Int64,1}
    # ... and `Psimplex` will store the corresponding points
    Psimplex = P[:, newidx][:,:]    #Array{Float64,2}

    # the second point must be seeken across those with coordinate `ax`
    #  grater than `off`
    Pselection = findall(x -> x > off, P[ax, :])

    for d = 1 : dim
        idxbase = findClosestPoint(Psimplex, P[:, Pselection])
        @assert !isnothing(idxbase) "firstDeWallSimplex:
            not able to determine first Delaunay Simplex"
        newidx = Pselection[idxbase]
        indices = [indices; newidx]
        Psimplex = [Psimplex P[:, newidx]]
        Pselection = [i for i = 1 : n if i ∉ indices]
    end

    # Correctness check
    radius, center = findRadius(Psimplex, true)
    for i = 1 : n
        @assert Lar.norm(center - P[:, i]) >= radius "firstDeWallSimplex:
            Unable to find first Simplex."
    end

    if DEBUG println("First Simplex = $indices") end

    return indices
end

firstDeWallSimplex (generic function with 1 method)

In [130]:
function recursiveDelaunayWall(
        P::Lar.Points,
        Pblack::Array{Float64},
        tetraDict::DataStructures.Dict{Array{Int64,1},Array{Float64,1}},
        AFL::Array{Array{Int64,1},1},
        ax::Int64,
        off::Float64,
        positive::Bool;
        DEBUG = false
    )::Lar.Cells

    #DEBUG = true

    dim, n = size(P)
    newaxis = mod(ax, dim) + 1

    if DEBUG println("Divide Plus/Minus $positive") end

    Psubset = findall(x -> (x > off) == positive, P[ax, :])
    blacklist = setdiff(unique([(keys(tetraDict)...)...]), Psubset)
    if !isempty(Pblack)
        Pblack = [Pblack P[:, blacklist]]
    else
        Pblack = P[:, blacklist]
    end

    if DEBUG println("Step In") end

    DT = delaunayWall(
            P[:, Psubset],
            newaxis,
            Pblack,
            [[findall(Psubset.==p)[1] for p in σ] for σ in AFL],
            Dict([
                [findall(Psubset.==p)[1] for p in k] => v
                for (k,v) in tetraDict
                    if k ⊆ Psubset
            ]),
            DEBUG = DEBUG
        )

    if DEBUG @show "Step Out with " DT end

    return [[Psubset[i] for i in σ] for σ in DT]
end


recursiveDelaunayWall (generic function with 1 method)

In [131]:
function updateAFL!(
        P::Lar.Points,
        newσ::Array{Array{Int64,1},1},
        AFLα::Array{Array{Int64,1},1},
        AFLplus::Array{Array{Int64,1},1},
        AFLminus::Array{Array{Int64,1},1},
        ax::Int64, off::Float64;
        DEBUG = false
    )::Bool

    for face in newσ
        inters = planarIntersection(P, face, ax, off)
        if inters == 0 # intersected by plane α
            updatelist!(AFLα, face)
        elseif inters == -1 # in NegHalfspace(α)
            updatelist!(AFLminus, face)
        elseif inters == 1 # in PosHalfspace(α)
            updatelist!(AFLplus, face)
        else
            return false
        end
    end

    if DEBUG @show AFLα AFLminus AFLplus end

    return true

end


updateAFL! (generic function with 1 method)

In [132]:
function updatelist!(list, element)::Bool
    if element ∈ list
        setdiff!(list, [element])
        return false
    else
        push!(list, element)
        return true
    end
end

updatelist! (generic function with 1 method)

In [133]:
function updateTetraDict!(
        P::Lar.Points,
        tetraDict::DataStructures.Dict{Array{Int64,1},Array{Float64,1}},
        AFL::Array{Array{Int64,1},1},
        σ::Array{Int64,1}
    )::Nothing
    for cell in AFL
        point = setdiff(σ, cell)
        @assert length(point) == 1 "updateTetraDict!: Error during update of TetraDict $σ, $cell"
        tetraDict[ cell ] = P[:, point[1]]
    end
end


updateTetraDict! (generic function with 1 method)

In [134]:
function alphaFilter(
        V::Lar.Points,
        DT = Array{Int64,1}[];
        digits=64
    )::DataStructures.SortedDict{}

    dim = size(V, 1)
    filtration = DataStructures.SortedDict{Array{Int64,1},Float64}()

    # 1 - Each point => alpha_char = 0.
    for i = 1 : size(V, 2)
        insert!(filtration, [i], 0.)
    end

    # 2 - Delaunay triangulation of ``V``
    if isempty(DT)
        DT = delaunayTriangulation(V)
    end

    n_upsimplex = length(DT)

    # 3 - process all upper simplex
    ind = 1
    for upper_simplex in DT
        if ind % 500000 == 0
            println(ind," simplices processed of ", n_upsimplex)
        end
        processuppersimplex(V,upper_simplex,filtration; digits = digits)
        ind = ind + 1
    end

    return filtration
end

alphaFilter (generic function with 2 methods)

In [135]:
function processuppersimplex(
        V::Lar.Points,
        up_simplex::Array{Int64,1},
        filtration::DataStructures.SortedDict{};
        digits=64
        )

    α_char = findRadius(V[:, up_simplex], digits=digits);
    insert!(filtration, up_simplex, α_char)

    d = length(up_simplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(up_simplex,d))
        for lowsimplex in newsimplex
            processlowsimplex(V, up_simplex, lowsimplex, filtration; digits=digits)
        end
    end
end

processuppersimplex (generic function with 1 method)

In [136]:
function processlowsimplex(
    V::Lar.Points,
    up_simplex::Array{Int64,1},
    lowsimplex::Array{Int64,1},
    filtration::DataStructures.SortedDict{};
    digits=64)

    α_char = findRadius(V[:, lowsimplex], digits=digits)
    point = V[:, setdiff(up_simplex, lowsimplex)]

    if vertexInCircumball(V[:, lowsimplex], α_char, point)
        filtration[lowsimplex] = filtration[up_simplex]

    elseif !haskey(filtration, lowsimplex)
        filtration[lowsimplex] = α_char

    end

    d = length(lowsimplex)-1
    if d > 1
        # It gives back combinations in natural order
        newsimplex = collect(Combinatorics.combinations(lowsimplex,d))
        for simplex in newsimplex
             processlowsimplex(V, lowsimplex, simplex, filtration, digits=digits)
        end
    end
end


processlowsimplex (generic function with 1 method)

In [137]:
function alphaSimplex(
        V::Lar.Points,
        filtration::DataStructures.SortedDict{},
        α_threshold::Float64
    )::Array{Lar.Cells,1}

    dim = size(V, 1)
    # [VV, EV, FV, ...]
    simplexCollection = [ Array{Array{Int64,1},1}() for i = 1 : dim+1 ]

    for (k, v) in filtration
        if v <= α_threshold
            push!(simplexCollection[length(k)], k)
        end
    end

    sort!.(simplexCollection)

    return simplexCollection
end

alphaSimplex (generic function with 1 method)

In [138]:
function delaunayTriangulation(V::Lar.Points)::Lar.Cells
    dim = size(V, 1)
    @assert dim > 0 "delaunayTriangulation: V do not contains points."
    @assert dim < 4 "delaunayTriangulation: Function not yet Programmed."

    if dim == 1
        vertices = vcat(V...)
        p = sortperm(vertices)
        upper_simplex = [[p[i],p[i+1]] for i=1:length(p)-1]

    elseif dim == 2
        vertices = convert(Array{Float64,2},V')
        points_map = Array{Int64,1}(collect(1:1:size(vertices)[1]))
        @assert size(vertices, 1) > 3
        upper_simplex = Triangle.basic_triangulation(vertices, points_map)

    elseif dim == 3
        upper_simplex = delaunayWall(V)
    end

    sort!.(upper_simplex)

    return sort(upper_simplex)
end

delaunayTriangulation (generic function with 1 method)

# Esempio 3D (utilizzando le funzioni del vecchio modulo)

In [139]:
using LinearAlgebraicRepresentation, ViewerGL
using BenchmarkTools
using Distributed
GL = ViewerGL
filename = "../examples/examples3D/OBJ/teapot.obj";

W, EVs, FVs = Lar.obj2lar(filename);
WW = [[i] for i = 1:size(W, 2)];
V, VV = Lar.apply(Lar.r(pi / 2, 0, 0), (W, WW)); #object rotated

points = convert(Lar.Points, V')
#=
GL.VIEW([
    GL.GLPoints(points)
    GL.GLAxis(GL.Point3d(-1, -1, -1), GL.Point3d(1, 1, 1))
]);
=#

529×3 Array{Float64,2}:
 20.2617   3.9956  22.3469
 19.2515  -1.1209  22.3469
 19.0837  -1.0495  23.0343
 20.0799   3.9956  23.0343
 19.2783  -1.1323  23.2634
 20.2908   3.9956  23.2634
 19.6742  -1.3008  23.0343
 20.72     3.9956  23.0343
 20.1104  -1.4864  22.3469
 21.1929   3.9956  22.3469
 16.4814  -5.2597  22.3469
 16.3523  -5.1306  23.0343
 16.5021  -5.2804  23.2634
  ⋮                
  8.8935   7.9146  24.3109
 10.2411  11.082   23.7435
 11.4517  13.9273  23.1761
 11.9771  15.162   22.3469
 10.2423   7.0118  24.3109
 12.6801   9.4496  23.7435
 14.87    11.6395  23.1761
 15.8203  12.5898  22.3469
 11.1451   5.663   24.3109
 14.3125   7.0107  23.7435
 17.1578   8.2213  23.1761
 18.3925   8.7466  22.3469

In [140]:
@btime filtration = alphaFilter(V); # 8.448 s (51918223 allocations: 4.07 GiB)

  7.510 s (51915163 allocations: 4.07 GiB)


In [141]:
@benchmark filtration = alphaFilter(V);

In [142]:
@code_warntype alphaFilter(V);

Variables
  #self#::Core.Compiler.Const(alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.Array, Main.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [143]:
filtration = alphaFilter(V);

In [144]:
@btime VV, EV, FV, TV = alphaSimplex(V, filtration, 3.7) # 2.156 ms (55 allocations: 205.47 KiB)

  2.228 ms (55 allocations: 205.47 KiB)


4-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [520], [521], [522], [523], [524], [525], [526], [527], [528], [529]]
 [[1, 2], [1, 3], [1, 4], [1, 8], [1, 10], [1, 76], [1, 77], [1, 82], [1, 472], [1, 473]  …  [523, 524], [523, 527], [523, 528], [524, 525], [524, 528], [524, 529], [525, 529], [526, 527], [527, 528], [528, 529]]
 [[1, 2, 3], [1, 2, 8], [1, 2, 10], [1, 2, 472], [1, 3, 4], [1, 3, 8], [1, 3, 472], [1, 4, 8], [1, 4, 77], [1, 4, 472]  …  [519, 522, 523], [520, 521, 524], [520, 523, 524], [521, 524, 525], [522, 523, 527], [522, 526, 527], [523, 524, 528], [523, 527, 528], [524, 525, 529], [524, 528, 529]]
 [[1, 2, 3, 8], [1, 2, 3, 472], [1, 2, 8, 10], [1, 3, 4, 8], [1, 3, 4, 472], [1, 4, 8, 77], [1, 4, 77, 529], [1, 4, 472, 473], [1, 4, 473, 529], [1, 8, 10, 76]  …  [459, 460, 518, 522], [459, 460, 522, 526], [460, 461, 464, 465], [460, 461, 465, 526], [460, 461, 518, 522], [460, 461, 522, 526], [460, 463, 464, 526], [460, 

In [145]:
@benchmark VV, EV, FV, TV = alphaSimplex(V, filtration, 3.7)

BenchmarkTools.Trial: 
  memory estimate:  205.47 KiB
  allocs estimate:  55
  --------------
  minimum time:     2.223 ms (0.00% GC)
  median time:      2.863 ms (0.00% GC)
  mean time:        3.080 ms (0.96% GC)
  maximum time:     11.913 ms (77.62% GC)
  --------------
  samples:          1618
  evals/sample:     1

In [146]:
@code_warntype alphaSimplex(V, filtration, 3.7)

Variables
  #self#::Core.Compiler.Const(alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #446::var"#446#447"
  dim::Int64
  simplexCollection::Array{Array{Array{Int64,1},1},1}
  @_8::Union{Nothing, Tuple{Pair{Array{Int64,1},Float64},DataStructures.SAIterationState}}
  k::Array{Int64,1}
  v::Float64
  @_11::Int64

Body::Array{_A,1} where _A
1 ─ %1  = Base.getproperty(Main.Lar, :Cells)::Any
│   %2  = Core.apply_type(Main.Array, %1, 1)::Type{Array{_A,1}} where _A
│         (dim = Main.size(V, 1))
│         (#446 = %new(Main.:(var"#446#447")))
│   %5  = #446::Core.Compiler.Const(var"#446#447"(), false)
│   %6  = (dim + 1)::Int64
│   %7  = (1:%6)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│   %8  = Base.Generator(%5, %7)::Core.Compiler.PartialStruct(Base.Generator{UnitRange{Int64},var"#446#447"}, Any[Core.Compiler.Const(var"#446#447"(), false), Core.C

In [147]:
VV, EV, FV, TV = alphaSimplex(V, filtration, 3.7)
#=
GL.VIEW([
    GL.GLGrid(V, EV, GL.COLORS[1], 0.6) # White
    GL.GLGrid(V, FV, GL.COLORS[2], 0.3) # Red
    GL.GLGrid(V, TV, GL.COLORS[3], 0.3) # Green
]);
=#
filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter =
    filter_key[sort(abs.(rand(Int, granular) .% length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

α=0.0
for α in reduced_filter
    @show α
    @btime VVV, EEV, FFV, TTV = alphaSimplex(V, filtration, α) # 772.066 μs (20 allocations: 19.34 KiB) in media
end

α = 1.6732420297499826
  788.349 μs (20 allocations: 19.34 KiB)
α = 1.7457423850614395
  832.472 μs (20 allocations: 19.34 KiB)
α = 1.796613669185222
  767.657 μs (20 allocations: 19.34 KiB)
α = 2.68722184082027
  738.399 μs (20 allocations: 19.34 KiB)
α = 2.9892045125543354
  768.914 μs (20 allocations: 19.34 KiB)
α = 3.8366361535836235
  748.743 μs (20 allocations: 19.34 KiB)
α = 7.815471691358947
  792.256 μs (20 allocations: 19.34 KiB)
α = 10.408317716047518
  746.379 μs (20 allocations: 19.34 KiB)
α = 11.20978520870777
  741.320 μs (20 allocations: 19.34 KiB)
α = 12.514114594018103
  764.527 μs (20 allocations: 19.34 KiB)
α = Inf
  788.107 μs (20 allocations: 19.34 KiB)


# Esempio 3D (utilizzando le funzioni del nuovo modulo)

In [148]:
using LinearAlgebraicRepresentation, ViewerGL
using BenchmarkTools
using Distributed
GL = ViewerGL
filename = "../examples/examples3D/OBJ/teapot.obj";

W, EVs, FVs = Lar.obj2lar(filename);
WW = [[i] for i = 1:size(W, 2)];
V, VV = Lar.apply(Lar.r(pi / 2, 0, 0), (W, WW)); #object rotated

points = convert(Lar.Points, V')
#=
GL.VIEW([
    GL.GLPoints(points)
    GL.GLAxis(GL.Point3d(-1, -1, -1), GL.Point3d(1, 1, 1))
]);
=#

529×3 Array{Float64,2}:
 20.2617   3.9956  22.3469
 19.2515  -1.1209  22.3469
 19.0837  -1.0495  23.0343
 20.0799   3.9956  23.0343
 19.2783  -1.1323  23.2634
 20.2908   3.9956  23.2634
 19.6742  -1.3008  23.0343
 20.72     3.9956  23.0343
 20.1104  -1.4864  22.3469
 21.1929   3.9956  22.3469
 16.4814  -5.2597  22.3469
 16.3523  -5.1306  23.0343
 16.5021  -5.2804  23.2634
  ⋮                
  8.8935   7.9146  24.3109
 10.2411  11.082   23.7435
 11.4517  13.9273  23.1761
 11.9771  15.162   22.3469
 10.2423   7.0118  24.3109
 12.6801   9.4496  23.7435
 14.87    11.6395  23.1761
 15.8203  12.5898  22.3469
 11.1451   5.663   24.3109
 14.3125   7.0107  23.7435
 17.1578   8.2213  23.1761
 18.3925   8.7466  22.3469

In [149]:
@btime filtration = AlphaStructures.alphaFilter(V); # 5.039 s (35330571 allocations: 3.80 GiB)

  5.835 s (33372481 allocations: 3.73 GiB)


In [150]:
@benchmark filtration = AlphaStructures.alphaFilter(V);

In [151]:
@code_warntype AlphaStructures.alphaFilter(V);

Variables
  #self#::Core.Compiler.Const(Main.AlphaStructures.alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.AlphaStructures.Array, Main.AlphaStructures.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [152]:
filtration = AlphaStructures.alphaFilter(V);

In [153]:
@btime VV, EV, FV, TV = AlphaStructures.alphaSimplex(V, filtration, 3.7) # 2.110 ms (55 allocations: 205.47 KiB)

  1.988 ms (55 allocations: 205.47 KiB)


4-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [520], [521], [522], [523], [524], [525], [526], [527], [528], [529]]
 [[1, 2], [1, 3], [1, 4], [1, 8], [1, 10], [1, 76], [1, 77], [1, 82], [1, 472], [1, 473]  …  [523, 524], [523, 527], [523, 528], [524, 525], [524, 528], [524, 529], [525, 529], [526, 527], [527, 528], [528, 529]]
 [[1, 2, 3], [1, 2, 8], [1, 2, 10], [1, 2, 472], [1, 3, 4], [1, 3, 8], [1, 3, 472], [1, 4, 8], [1, 4, 77], [1, 4, 472]  …  [519, 522, 523], [520, 521, 524], [520, 523, 524], [521, 524, 525], [522, 523, 527], [522, 526, 527], [523, 524, 528], [523, 527, 528], [524, 525, 529], [524, 528, 529]]
 [[1, 2, 3, 8], [1, 2, 3, 472], [1, 2, 8, 10], [1, 3, 4, 8], [1, 3, 4, 472], [1, 4, 8, 77], [1, 4, 77, 529], [1, 4, 472, 473], [1, 4, 473, 529], [1, 8, 10, 76]  …  [459, 460, 518, 522], [459, 460, 522, 526], [460, 461, 464, 465], [460, 461, 465, 526], [460, 461, 518, 522], [460, 461, 522, 526], [460, 463, 464, 526], [460, 

In [154]:
@benchmark VV, EV, FV, TV = AlphaStructures.alphaSimplex(V, filtration, 3.7)

BenchmarkTools.Trial: 
  memory estimate:  205.47 KiB
  allocs estimate:  55
  --------------
  minimum time:     1.951 ms (0.00% GC)
  median time:      2.952 ms (0.00% GC)
  mean time:        3.662 ms (0.90% GC)
  maximum time:     56.000 ms (0.00% GC)
  --------------
  samples:          1365
  evals/sample:     1

In [155]:
@code_warntype AlphaStructures.alphaSimplex(V, filtration, 3.7)

Variables
  #self#::Core.Compiler.Const(Main.AlphaStructures.alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #12::Main.AlphaStructures.var"#12#14"{Array{Float64,2},SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering},Float64}

Body::Array{Array{Array{Int64,1},1},1}
1 ─ %1  = LinearAlgebraicRepresentation.Cells::Core.Compiler.Const(Array{Array{Int64,1},1}, false)
│   %2  = Core.apply_type(Main.AlphaStructures.Array, %1, 1)::Core.Compiler.Const(Array{Array{Array{Int64,1},1},1}, false)
│   %3  = Main.AlphaStructures.:(var"#12#14")::Core.Compiler.Const(Main.AlphaStructures.var"#12#14", false)
│   %4  = Core.typeof(V)::Core.Compiler.Const(Array{Float64,2}, false)
│   %5  = Core.typeof(filtration)::Core.Compiler.Const(SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}, false)
│   %6  = Core.typeof(α_threshold)::Core.Compiler.Const(Float64, false)
│   %7  = Core.apply_type(%3, %

In [156]:
VV, EV, FV, TV = AlphaStructures.alphaSimplex(V, filtration, 3.7)
#=
GL.VIEW([
    GL.GLGrid(V, EV, GL.COLORS[1], 0.6) # White
    GL.GLGrid(V, FV, GL.COLORS[2], 0.3) # Red
    GL.GLGrid(V, TV, GL.COLORS[3], 0.3) # Green
]);
=#
filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter =
    filter_key[sort(abs.(rand(Int, granular) .% length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

α=0.0
for α in reduced_filter
    @show α
    @btime VVV, EEV, FFV, TTV = AlphaStructures.alphaSimplex(V, filtration, α) # 685.523 μs (20 allocations: 19.34 KiB) in media
end

α = 1.265860571903555
  707.758 μs (20 allocations: 19.34 KiB)
α = 1.2780301457518404
  698.427 μs (20 allocations: 19.34 KiB)
α = 1.3983413782049077
  762.883 μs (20 allocations: 19.34 KiB)
α = 1.5366627126666408
  719.961 μs (20 allocations: 19.34 KiB)
α = 1.871920996779934
  714.646 μs (20 allocations: 19.34 KiB)
α = 2.5996203915013227
  1.094 ms (20 allocations: 19.34 KiB)
α = 2.8424030022629454
  713.759 μs (20 allocations: 19.34 KiB)
α = 3.424469084074446
  685.758 μs (20 allocations: 19.34 KiB)
α = 3.531274098473569
  754.817 μs (20 allocations: 19.34 KiB)
α = 12.466449032742563
  734.776 μs (20 allocations: 19.34 KiB)
α = Inf
  680.355 μs (20 allocations: 19.34 KiB)


# Esempio 2D (utilizzando le funzioni del vecchio modulo)

In [157]:
"""
    pointsRand(V, VV, n, m)

Generate random points inside and otuside `(V, VV)`.
"""
function pointsRand(
        V::Lar.Points, EV::Lar.Cells, n = 1000, m = 0
    )::Tuple{Lar.Points, Lar.Points, Lar.Cells, Lar.Cells}
    classify = Lar.pointInPolygonClassification(V, EV)
    Vi = [0;0]
    Ve = [0;0]
    k1 = 0
    k2 = 0
    while k1 < n || k2 < m
        queryPoint = [rand();rand()]
        inOut = classify(queryPoint)

        if k1 < n && inOut == "p_in"
            Vi = hcat(Vi, queryPoint)
            k1 = k1 + 1;
        end
        if k2 < m && inOut == "p_out"
            Ve = hcat(Ve, queryPoint)
            k2 = k2 + 1;
        end
    end
    VVi = [[i] for i = 1 : n]
    VVe = [[i] for i = 1 : m]
    return Vi[:,2:end], Ve[:,2:end], VVi, VVe
end

filename = "../examples/examples2D/svg_files/Lar2.svg";

V,EV = Lar.svg2lar(filename);

Vi, Ve, VVi, VVe = pointsRand(V, EV, 1000, 10000);

#=
GL.VIEW([
 	GL.GLGrid(Vi, VVi, GL.COLORS[1], 1)
 	GL.GLGrid(Ve, VVe, GL.COLORS[12], 1)
])
=#

In [158]:
@btime filtration = alphaFilter(Vi); # 38.343 ms (371714 allocations: 25.58 MiB)

  37.081 ms (370628 allocations: 25.51 MiB)


In [159]:
@benchmark filtration = alphaFilter(Vi);

In [160]:
@code_warntype alphaFilter(Vi);

Variables
  #self#::Core.Compiler.Const(alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.Array, Main.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [161]:
filtration = alphaFilter(Vi);

In [162]:
@btime VV,EV,FV = alphaSimplex(Vi, filtration, 0.02) # 744.878 μs (41 allocations: 135.13 KiB)

  669.334 μs (41 allocations: 134.94 KiB)


3-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [991], [992], [993], [994], [995], [996], [997], [998], [999], [1000]]
 [[1, 95], [1, 379], [1, 795], [1, 826], [2, 149], [2, 150], [2, 194], [2, 231], [2, 526], [3, 4]  …  [951, 966], [952, 975], [955, 958], [963, 983], [969, 997], [971, 975], [974, 987], [977, 979], [984, 988], [994, 997]]
 [[1, 95, 795], [1, 95, 826], [1, 379, 795], [1, 379, 826], [2, 149, 231], [2, 149, 526], [2, 150, 194], [2, 150, 526], [2, 194, 231], [3, 4, 405]  …  [840, 866, 872], [852, 908, 919], [858, 867, 976], [869, 913, 918], [874, 896, 939], [883, 894, 917], [889, 923, 968], [897, 899, 945], [899, 935, 945], [907, 932, 986]]

In [163]:
@benchmark VV,EV,FV = alphaSimplex(Vi, filtration, 0.02)

BenchmarkTools.Trial: 
  memory estimate:  134.94 KiB
  allocs estimate:  41
  --------------
  minimum time:     661.807 μs (0.00% GC)
  median time:      1.070 ms (0.00% GC)
  mean time:        1.262 ms (1.79% GC)
  maximum time:     15.527 ms (87.24% GC)
  --------------
  samples:          3948
  evals/sample:     1

In [164]:
@code_warntype alphaSimplex(Vi, filtration, 0.02)

Variables
  #self#::Core.Compiler.Const(alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #446::var"#446#447"
  dim::Int64
  simplexCollection::Array{Array{Array{Int64,1},1},1}
  @_8::Union{Nothing, Tuple{Pair{Array{Int64,1},Float64},DataStructures.SAIterationState}}
  k::Array{Int64,1}
  v::Float64
  @_11::Int64

Body::Array{_A,1} where _A
1 ─ %1  = Base.getproperty(Main.Lar, :Cells)::Any
│   %2  = Core.apply_type(Main.Array, %1, 1)::Type{Array{_A,1}} where _A
│         (dim = Main.size(V, 1))
│         (#446 = %new(Main.:(var"#446#447")))
│   %5  = #446::Core.Compiler.Const(var"#446#447"(), false)
│   %6  = (dim + 1)::Int64
│   %7  = (1:%6)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│   %8  = Base.Generator(%5, %7)::Core.Compiler.PartialStruct(Base.Generator{UnitRange{Int64},var"#446#447"}, Any[Core.Compiler.Const(var"#446#447"(), false), Core.C

In [165]:
VV,EV,FV = alphaSimplex(Vi, filtration, 0.02)
points = [[p] for p in VV]
faces = [[f] for f in FV]
edges = [[e] for e in EV]
#GL.VIEW(
#    GL.GLExplode(Vi, [edges; faces], 1.5, 1.5, 1.5, 99, 1)
# );

filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter = filter_key[sort(abs.(rand(Int, granular).%length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

#
# Arlecchino's Lar
#
#
α = 0.0
for α in reduced_filter
    @show α
    @btime VV,EV,FV = alphaSimplex(Vi, filtration, α) # 315.970 μs (18 allocations: 21.00 KiB) in media
    #VV,EV,FV = alphaSimplex(Vi, filtration, α)

    #=
    GL.VIEW(
        GL.GLExplode(
            Vi,
            [[[f] for f in FV]; [[e] for e in EV]],
            1., 1., 1.,	# Explode Ratio
            99, 1		# Colors
        )
    )
    =#
    
end
#
#
# Appearing Colors
#

reduced_filter = [
    0.002;	0.003;	0.004;	0.005;  0.006;
    0.007;	0.008;	0.009;	0.010;	0.013;
    0.015;	0.020;	0.050;	1.000
]

i=2
for i = 2 : length(reduced_filter)
    @btime VV0, EV0, FV0 = alphaSimplex(Vi, filtration, reduced_filter[i-1])# 337.409 μs (36 allocations: 42.03 KiB) in media
    @btime VV,  EV,  FV  = alphaSimplex(Vi, filtration, reduced_filter[i])# 297.250 μs (32 allocations: 30.86 KiB) in media
    
    #=EV0mesh = GL.GLGrid(Vi, EV0)
    FV0mesh = GL.GLGrid(Vi, FV0)
    EVmesh = GL.GLGrid(Vi, setdiff(EV, EV0), GL.COLORS[2], 1)
    FVmesh = GL.GLGrid(Vi, setdiff(FV, FV0), GL.COLORS[7], 1)
    GL.VIEW([EV0mesh; FV0mesh; EVmesh; FVmesh])=#
end

α = 0.0018494497798915769
  261.513 μs (18 allocations: 21.00 KiB)
α = 0.00253210014694557
  259.700 μs (18 allocations: 21.00 KiB)
α = 0.0028012478588421755
  260.478 μs (18 allocations: 21.00 KiB)
α = 0.0034753877675220367
  259.635 μs (18 allocations: 21.00 KiB)
α = 0.003598361131796303
  258.062 μs (18 allocations: 21.00 KiB)
α = 0.006173783133085484
  262.677 μs (18 allocations: 21.00 KiB)
α = 0.0068323061765421635
  260.661 μs (18 allocations: 21.00 KiB)
α = 0.006963655998767298
  260.080 μs (18 allocations: 21.00 KiB)
α = 0.009613007777398961
  259.491 μs (18 allocations: 21.00 KiB)
α = 0.20729729402455183
  260.211 μs (18 allocations: 21.00 KiB)
α = 332.50575236914773
  262.542 μs (18 allocations: 21.00 KiB)
  279.896 μs (30 allocations: 26.53 KiB)
  311.782 μs (36 allocations: 42.03 KiB)
  280.894 μs (30 allocations: 26.53 KiB)
  310.309 μs (36 allocations: 42.03 KiB)
  296.164 μs (30 allocations: 26.53 KiB)
  314.567 μs (36 allocations: 42.03 KiB)
  279.645 μs (30 allocations

# Esempio 2D (utilizzando le funzioni del nuovo modulo)

In [166]:
"""
    pointsRand(V, VV, n, m)

Generate random points inside and otuside `(V, VV)`.
"""
function pointsRand(
        V::Lar.Points, EV::Lar.Cells, n = 1000, m = 0
    )::Tuple{Lar.Points, Lar.Points, Lar.Cells, Lar.Cells}
    classify = Lar.pointInPolygonClassification(V, EV)
    Vi = [0;0]
    Ve = [0;0]
    k1 = 0
    k2 = 0
    while k1 < n || k2 < m
        queryPoint = [rand();rand()]
        inOut = classify(queryPoint)

        if k1 < n && inOut == "p_in"
            Vi = hcat(Vi, queryPoint)
            k1 = k1 + 1;
        end
        if k2 < m && inOut == "p_out"
            Ve = hcat(Ve, queryPoint)
            k2 = k2 + 1;
        end
    end
    VVi = [[i] for i = 1 : n]
    VVe = [[i] for i = 1 : m]
    return Vi[:,2:end], Ve[:,2:end], VVi, VVe
end

filename = "../examples/examples2D/svg_files/Lar2.svg";

V,EV = Lar.svg2lar(filename);

Vi, Ve, VVi, VVe = pointsRand(V, EV, 1000, 10000);

#=
GL.VIEW([
 	GL.GLGrid(Vi, VVi, GL.COLORS[1], 1)
 	GL.GLGrid(Ve, VVe, GL.COLORS[12], 1)
])
=#

In [167]:
@btime filtration = AlphaStructures.alphaFilter(Vi); # 30.037 ms (283750 allocations: 24.19 MiB)

  29.011 ms (282384 allocations: 24.18 MiB)


In [168]:
@benchmark filtration = AlphaStructures.alphaFilter(Vi);

In [169]:
@code_warntype AlphaStructures.alphaFilter(Vi); 

Variables
  #self#::Core.Compiler.Const(Main.AlphaStructures.alphaFilter, false)
  V::Array{Float64,2}

Body::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
1 ─ %1 = Core.apply_type(Main.AlphaStructures.Array, Main.AlphaStructures.Int64, 1)::Core.Compiler.Const(Array{Int64,1}, false)
│   %2 = Base.getindex(%1)::Array{Array{Int64,1},1}
│   %3 = (#self#)(V, %2)::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
└──      return %3


In [170]:
filtration = AlphaStructures.alphaFilter(Vi); 

In [171]:
@btime VV,EV,FV = AlphaStructures.alphaSimplex(Vi, filtration, 0.02) # 690.470 μs (41 allocations: 135.00 KiB)

  658.656 μs (41 allocations: 134.94 KiB)


3-element Array{Array{Array{Int64,1},1},1}:
 [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [991], [992], [993], [994], [995], [996], [997], [998], [999], [1000]]
 [[1, 54], [1, 186], [1, 224], [1, 289], [1, 387], [1, 801], [2, 166], [2, 389], [2, 806], [2, 985]  …  [933, 960], [939, 944], [942, 956], [945, 966], [961, 994], [965, 994], [966, 967], [969, 992], [970, 973], [978, 999]]
 [[1, 54, 387], [1, 54, 801], [1, 186, 224], [1, 186, 801], [1, 224, 289], [1, 289, 387], [2, 166, 806], [2, 166, 985], [2, 389, 806], [2, 389, 985]  …  [845, 890, 995], [858, 939, 944], [871, 920, 942], [871, 920, 964], [871, 924, 927], [871, 927, 964], [872, 906, 996], [918, 966, 967], [925, 939, 944], [930, 942, 956]]

In [172]:
@benchmark VV,EV,FV = AlphaStructures.alphaSimplex(Vi, filtration, 0.02)

BenchmarkTools.Trial: 
  memory estimate:  134.94 KiB
  allocs estimate:  41
  --------------
  minimum time:     661.880 μs (0.00% GC)
  median time:      954.317 μs (0.00% GC)
  mean time:        1.449 ms (1.65% GC)
  maximum time:     42.671 ms (0.00% GC)
  --------------
  samples:          3448
  evals/sample:     1

In [173]:
@code_warntype AlphaStructures.alphaSimplex(Vi, filtration, 0.02)

Variables
  #self#::Core.Compiler.Const(Main.AlphaStructures.alphaSimplex, false)
  V::Array{Float64,2}
  filtration::SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}
  α_threshold::Float64
  #12::Main.AlphaStructures.var"#12#14"{Array{Float64,2},SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering},Float64}

Body::Array{Array{Array{Int64,1},1},1}
1 ─ %1  = LinearAlgebraicRepresentation.Cells::Core.Compiler.Const(Array{Array{Int64,1},1}, false)
│   %2  = Core.apply_type(Main.AlphaStructures.Array, %1, 1)::Core.Compiler.Const(Array{Array{Array{Int64,1},1},1}, false)
│   %3  = Main.AlphaStructures.:(var"#12#14")::Core.Compiler.Const(Main.AlphaStructures.var"#12#14", false)
│   %4  = Core.typeof(V)::Core.Compiler.Const(Array{Float64,2}, false)
│   %5  = Core.typeof(filtration)::Core.Compiler.Const(SortedDict{Array{Int64,1},Float64,Base.Order.ForwardOrdering}, false)
│   %6  = Core.typeof(α_threshold)::Core.Compiler.Const(Float64, false)
│   %7  = Core.apply_type(%3, %

In [ ]:
VV,EV,FV = AlphaStructures.alphaSimplex(Vi, filtration, 0.02)
points = [[p] for p in VV]
faces = [[f] for f in FV]
edges = [[e] for e in EV]
#GL.VIEW(
#    GL.GLExplode(Vi, [edges; faces], 1.5, 1.5, 1.5, 99, 1)
# );

filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter = filter_key[sort(abs.(rand(Int, granular).%length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

#
# Arlecchino's Lar
#
#
α = 0.0
for α in reduced_filter
    @show α
    @btime VV,EV,FV = AlphaStructures.alphaSimplex(Vi, filtration, α) # 255.389 μs (18 allocations: 21.00 KiB) in media
    #VV,EV,FV = alphaSimplex(Vi, filtration, α)

    #=
    GL.VIEW(
        GL.GLExplode(
            Vi,
            [[[f] for f in FV]; [[e] for e in EV]],
            1., 1., 1.,	# Explode Ratio
            99, 1		# Colors
        )
    )
    =#
    
end
#
#
# Appearing Colors
#

reduced_filter = [
    0.002;	0.003;	0.004;	0.005;  0.006;
    0.007;	0.008;	0.009;	0.010;	0.013;
    0.015;	0.020;	0.050;	1.000
]

i=2
for i = 2 : length(reduced_filter)
    @btime VV0, EV0, FV0 = AlphaStructures.alphaSimplex(Vi, filtration, reduced_filter[i-1]) # 292.120 μs (35 allocations: 40.78 KiB) in media
    @btime VV,  EV,  FV  = AlphaStructures.alphaSimplex(Vi, filtration, reduced_filter[i]) # 266.537 μs (30 allocations: 26.45 KiB) in media
    
    #=EV0mesh = GL.GLGrid(Vi, EV0)
    FV0mesh = GL.GLGrid(Vi, FV0)
    EVmesh = GL.GLGrid(Vi, setdiff(EV, EV0), GL.COLORS[2], 1)
    FVmesh = GL.GLGrid(Vi, setdiff(FV, FV0), GL.COLORS[7], 1)
    GL.VIEW([EV0mesh; FV0mesh; EVmesh; FVmesh])=#
end

α = 0.001143377007748084
  242.695 μs (18 allocations: 21.00 KiB)
α = 0.0030274175860053477
  243.389 μs (18 allocations: 21.00 KiB)
α = 0.004373761624185768
  244.232 μs (18 allocations: 21.00 KiB)
α = 0.004670240769166227
  244.873 μs (18 allocations: 21.00 KiB)
α = 0.005587395125027122
  246.215 μs (18 allocations: 21.00 KiB)
α = 0.005749989149830111
  242.283 μs (18 allocations: 21.00 KiB)
α = 0.0062719953885048765
  242.793 μs (18 allocations: 21.00 KiB)
α = 0.007293151915619161
  243.567 μs (18 allocations: 21.00 KiB)
α = 0.007782234440256091
  243.629 μs (18 allocations: 21.00 KiB)
α = 3.4918259237858393
  243.032 μs (18 allocations: 21.00 KiB)
α = 11.214595116575394
  243.000 μs (18 allocations: 21.00 KiB)
  265.884 μs (31 allocations: 26.72 KiB)
  292.367 μs (36 allocations: 41.97 KiB)
  261.878 μs (31 allocations: 26.72 KiB)
  290.715 μs (36 allocations: 41.97 KiB)
  263.283 μs (31 allocations: 26.72 KiB)
  291.708 μs (36 allocations: 41.97 KiB)
  263.455 μs (31 allocations: 